### Data Generator

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import os

In [2]:
train_idg = image.ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True
)

val_idg = image.ImageDataGenerator(
    rescale=1./255
)

In [3]:
train_generator = train_idg.flow_from_directory(
    '../Dataset/train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 273 images belonging to 2 classes.


In [4]:
val_generator = val_idg.flow_from_directory(
    '../Dataset/test',
    target_size=(224,224),
    batch_size=24,
    class_mode='binary'
)

Found 68 images belonging to 2 classes.


In [5]:
from keras.applications import VGG16,ResNet50
from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam

### Vgg16 Model

In [6]:
model = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

In [7]:
av1 = GlobalAveragePooling2D()(model.output)
f1 = Flatten()(av1)
drop1 = Dropout(0.5)(f1)
d1 = Dense(256,activation='relu')(drop1)
d2 = Dropout(0.5)(d1)
out = Dense(1,activation='sigmoid')(d2)

In [8]:
vgg = Model(model.input,out)
print(len(vgg.layers))

25


In [9]:
for i in range(17): 
    vgg.layers[i].trainable = False
vgg.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

for i in range(169):
    resnet50.layers[i].trainable = False
    
resnet50.summary()

In [10]:
adam = Adam(lr=0.001)
vgg.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

- Train Model

In [11]:
from keras.callbacks import ModelCheckpoint
import glob

In [12]:
if not os.path.exists('../Weights/Vgg'):
    os.mkdir('../Weights/Vgg')
    print('Directory created')
else:
    print('Already Created')

Already Created


In [13]:
files = glob.glob('../Weights/Vgg/*')
for f in files:
    os.remove(f)

In [14]:
checkpoint = ModelCheckpoint(
    '../Weights/Vgg/model_{epoch}_{val_accuracy:.2f}.h5',
    monitor='val_accuracy', 
    verbose=1, 
    mode='auto'
)

In [15]:
hist = vgg.fit(
    train_generator,
    steps_per_epoch=9,
    epochs=10,
    validation_data=val_generator,
    validation_steps=3,
    callbacks=checkpoint
)

Epoch 1/10
9/9 [==============================] - ETA: 0s - loss: 0.5238 - accuracy: 0.7179 
Epoch 00001: saving model to ../Weights/Vgg/model_1_0.75.h5
9/9 [==============================] - 304s 34s/step - loss: 0.5238 - accuracy: 0.7179 - val_loss: 0.5322 - val_accuracy: 0.7500
Epoch 2/10
9/9 [==============================] - ETA: 0s - loss: 0.2694 - accuracy: 0.8974 
Epoch 00002: saving model to ../Weights/Vgg/model_2_0.85.h5
9/9 [==============================] - 182s 20s/step - loss: 0.2694 - accuracy: 0.8974 - val_loss: 0.5789 - val_accuracy: 0.8529
Epoch 3/10
9/9 [==============================] - ETA: 0s - loss: 0.2344 - accuracy: 0.9158 
Epoch 00003: saving model to ../Weights/Vgg/model_3_0.84.h5
9/9 [==============================] - 171s 19s/step - loss: 0.2344 - accuracy: 0.9158 - val_loss: 0.6444 - val_accuracy: 0.8382
Epoch 4/10
9/9 [==============================] - ETA: 0s - loss: 0.1874 - accuracy: 0.9414 
Epoch 00004: saving model to ../Weights/Vgg/model_4_0.96.h5
9

In [16]:
x,y=val_generator.next()

In [17]:
vgg.evaluate(x,y)

1/1 [==============================] - 0s 0s/step - loss: 0.0013 - accuracy: 1.0000


[0.0013214204227551818, 1.0]

- Save Model History

In [18]:
import numpy as np

In [19]:
if not os.path.exists('../Model_history'):
    os.mkdir('../Model_history')
    print('Directory created')
else:
    print('Already Created')

Already Created


In [20]:
np.save('../Model_history/vgg_file.npy',hist.history)
read_dictionary = np.load('../Model_history/vgg_file.npy',allow_pickle='TRUE').item()
print(read_dictionary.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
